<a href="https://colab.research.google.com/github/awhitehouse1/DS4002-Project-2/blob/main/Scripts/Final_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
# Time series data - 2020 with merged covid, gdp, democracy scores

# Load the datasets
covid_data = pd.read_csv("owid-covid-data.csv")
gdp_data = pd.read_csv("gdp_per_capita.csv", delimiter=',', skiprows=3, on_bad_lines='warn', keep_default_na=True)
democracy_data = pd.read_csv("democracy-index-eiu.csv")

# Convert the 'Date' column in the COVID dataset to datetime format
covid_data['date'] = pd.to_datetime(covid_data['date'])

# Filter the COVID data for the date range from January 22, 2020, to January 22, 2022
start_date = '2020-01-22'
end_date = '2022-01-22'
covid_filtered = covid_data[(covid_data['date'] >= start_date) & (covid_data['date'] <= end_date)]

# Create 'Year' and 'Month' columns from the 'Date' column
covid_filtered['Year'] = covid_filtered['date'].dt.year
covid_filtered['Month'] = covid_filtered['date'].dt.month

covid_filtered = covid_filtered[['Year', 'Month', 'total_deaths_per_million', 'new_deaths_per_million', 'location', 'date' ]]

covid_filtered = covid_filtered.rename(columns={'location': 'Country', 'date': 'Date'})


# Melt the GDP data to have a 'Year' column
gdp_melted = gdp_data.melt(id_vars='Country Name', value_vars=['2020', '2021', '2022'],
                           var_name='Year', value_name='GDP_per_Capita')


# Convert the 'Year' column to numeric and rename 'Country Name' column
gdp_melted['Year'] = pd.to_numeric(gdp_melted['Year'])
gdp_melted = gdp_melted.rename(columns={'Country Name': 'Country'})

# Filter GDP data for the years 2020 to 2022
gdp_filtered = gdp_melted[gdp_melted['Year'].isin([2020, 2021, 2022])]


# Rename columns in the democracy dataset
democracy_data = democracy_data.rename(columns={'Entity': 'Country'})

# Filter democracy data for the years 2020 to 2022
democracy_filtered = democracy_data[democracy_data['Year'].isin([2020, 2021, 2022])]

# Merge the COVID dataset with the GDP dataset on 'Country' and 'Year'
merged_data = pd.merge(covid_filtered, gdp_filtered, how='inner', on=['Country', 'Year'])


# Merge the above result with democracy data on 'Country' and 'Year'
final_merged_data = pd.merge(merged_data, democracy_filtered, how='inner', on=['Country', 'Year'])

final_merged_data = final_merged_data.drop(columns = ['Code'])

final_merged_data = final_merged_data[final_merged_data['Country'] != 'World']

final_merged_data['Country'] = final_merged_data['Country'].astype('category')
final_merged_data['Country_Codes'] = final_merged_data['Country'].cat.codes

# Export the final merged DataFrame to a CSV file
final_merged_data.to_csv('NEW_time_series_data.csv', index=False)




<ipython-input-2-63e660033235>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Year'] = covid_filtered['date'].dt.year
<ipython-input-2-63e660033235>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Month'] = covid_filtered['date'].dt.month
